In [0]:
# Iniciamos sesion en SPARK
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) **PARTICIONES**
Spark organiza los datos en particiones, considerándolas divisiones lógicas de los datos entre los nodos del clúster.

Cada una de las particiones va a llevar asociada una tarea de ejecución, de manera que a más particiones, mayor paralelización del proceso.

Seguidamente se muestra un código para trabajar con las particiones:

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5])
print(rdd.getNumPartitions()) #8

8


In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)
print(rdd.getNumPartitions()) #2

2


In [0]:
rddE = sc.textFile("/FileStore/tables/empleados.txt") # Usamos el fichero que se cargó en un notebook anterior: empleados.txt
print(rddE.getNumPartitions()) #2

2


In [0]:
rddE = sc.textFile("/FileStore/tables/empleados.txt", 3)
rddE.getNumPartitions() # 3

Out[11]: 3

La mayoría de operaciones/transformaciones/acciones que trabajan con los datos admiten un parámetro extra indicando la cantidad de particiones con las que queremos trabajar!!!

### *MapPartitions*
A diferencia de la transformación map que se invoca por cada elemento del RDD/DataSet, *mapPartitions* se llama por cada partición.

La función que recibe como parámetro recogerá como entrada un iterador con los elementos de cada partición:

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)

def f(iterator): yield sum(iterator)
resultadoRdd = rdd.mapPartitions(f)
resultadoRdd.collect()  # [6, 15]


Out[13]: [6, 15]

In [0]:
resultadoRdd2 = rdd.mapPartitions(lambda iterator: [list(iterator)])
resultadoRdd2.collect() # [[1, 1, 2, 2], [3, 3, 4, 5]]

Out[14]: [[1, 1, 2, 2], [3, 3, 4, 5]]

En el caso anterior, se ha realizado una división de los datos en dos particiones, la primera con [1, 1, 2, 2] y la otra con [3, 3, 4, 5], y de ahí el resultado de sumar sus elementos es [6, 15]

#### *mapPartitionsWithIndex*

De forma similar al caso anterior, pero ahora *mapPartitionsWithIndex* recibe una función cuyos parámetros son el índice de la partición y el iterador con los datos de la misma:

In [0]:
def mpwi(indice, iterador):
    return [(indice, list(iterador))]

resultadoRdd = rdd.mapPartitionsWithIndex(mpwi)
resultadoRdd.collect()
# [(0, [1, 1, 2, 2]), (1, [3, 3, 4, 5])]

Out[15]: [(0, [1, 1, 2, 2]), (1, [3, 3, 4, 5])]

**Modificando las particiones**.  

Podemos modificar la cantidad de particiones mediante dos transformaciones wide: *coalesce* y *repartition*.

Mediante coalesce podemos obtener un nuevo RDD con la cantidad de particiones a reducir:


In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
print(rdd.getNumPartitions()) #3

3


In [0]:
rdd1p = rdd.coalesce(2)
print(rdd1p.getNumPartitions()) #2

2


En cambio, mediante *repartition* podemos obtener un nuevo RDD con la cantidad exacta de particiones deseadas (al reducir las particiones, repartition realiza un *shuffle* para redistribuir los datos. Por lo tanto, si queremos reducir la cantidad de particiones, es más eficiente utilizar *coalesce*):

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
print(rdd.getNumPartitions()) # 3

3


In [0]:
rdd2p = rdd.repartition(2)
print(rdd2p.getNumPartitions()) # 2

2
